### Imports

In [1]:
import sys
sys.path.append("../../utils")
from definitions import *

/Users/nvdenisov2002/study/8-semester/diploma/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def convert_works_to_txt(year):
    base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
    base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

    processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
    processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)

    def convert_doc_to_txt(filename):
        subprocess.run(["textutil", "-convert", "txt", "-output", processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix(), filename])

    def convert_pdf_to_txt(filename):
        subprocess.run(["pdftotext", filename.as_posix(), processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix()])

    for ext in [".doc", ".docx", ".DOCX"]:
        for file in tqdm(base_diplomas_dir_path.glob(f"*{ext}"), desc="Docs..."):
            convert_doc_to_txt(file)

    
    for file in tqdm(base_diplomas_dir_path.glob(f"*.pdf"), desc="Pdfs..."):
        convert_pdf_to_txt(file)

In [13]:
years = [2022, 2021, 2020]

In [14]:
for year in tqdm(years, desc="Years..."):
    convert_works_to_txt(year)

Docs...: 25it [00:02,  8.51it/s]00:00<?, ?it/s]


### Convert .doc to .txt

In [4]:
base_spbu_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/")
base_diplomas_dir_path = base_spbu_dir_path.joinpath("work") 

exts = defaultdict(int)
for file in base_diplomas_dir_path.glob("*"):
    exts[file.suffix] += 1
print(exts)

for ext in exts:
    for file in base_diplomas_dir_path.glob(f"*{ext}"):
        if ext != ".pages":
            break

processed_work_ext_doc_dir_path = base_spbu_dir_path.joinpath("work_txt/")
processed_work_ext_doc_dir_path.mkdir(exist_ok=True, parents=True)
def convert_doc_to_txt(filename):
    subprocess.run(["textutil", "-convert", "txt", "-output", processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix(), filename])

# for ext in [".doc", ".docx", ".DOCX"]:
#     for file in base_diplomas_dir_path.glob(f"*{ext}"):
#         convert_doc_to_txt(file)

defaultdict(<class 'int'>, {'.txt': 1654, '.pdf': 881, '.docx': 722, '.doc': 29, '.jpg': 4, '.spv': 1, '.jpeg': 7, '.pages': 1, '.mp4': 1, '.DOCX': 1, '.xlsx': 2, '.rtf': 1, '.odt': 1, '.docm': 1, '.JPG': 1, '.PDF': 1})


In [8]:
def convert_pdf_to_txt(filename):
    subprocess.run(["pdftotext", filename.as_posix(), processed_work_ext_doc_dir_path.joinpath(f"{filename.stem}.txt").as_posix()])

for file in base_diplomas_dir_path.glob(f"*.pdf"):
    convert_pdf_to_txt(file)

Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Warning: Mismatch between font type and embedded font file
Syntax Error: Document stream is empty
Syntax Warning: Invalid Font Weight
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warning: Bad annotation destination
Syntax Warnin

### Process diplomas

In [3]:
RAW_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/work/")
PROCESSED_DIPLOMAS_DIR_PATH = ARTIFACTS_DIR_PATH.joinpath(f"parsing/diplomas/spbu/{year}/stupid_filtered_work")
PROCESSED_DIPLOMAS_DIR_PATH.mkdir(exist_ok=True, parents=True)

In [3]:
num_ok, num_all = 0, 0
for file in RAW_DIPLOMAS_DIR_PATH.glob("*.txt"):
    with open(file, "r") as f:
        text = f.read()
    if "выпускная квалификационная работа" in text.lower() and "отзыв" not in text.lower():
        with open(PROCESSED_DIPLOMAS_DIR_PATH.joinpath(f"{file.stem}.txt"), "w") as f:
            f.write(text)
        print(text[:10])
        num_ok += 1
    num_all += 1
print(num_ok, num_all)

Санкт-Пете
          
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
ФГОУ ВО «С
Санкт-Пете
Санкт-Пете
Санкт-Пете
САНКТ-ПЕТЕ
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете
Санкт-Пете

САНКТ-ПЕТ
Санкт-Пете
Санкт-Пете
23 143


### Split into smaller directories

In [3]:
with open(ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/ids.json"), "r") as f:
    ids = json.load(f)

In [5]:
len(ids)

2328

In [7]:
old_dir_path = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/work_doc")
new_dir_path_1 = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/work_doc_1")
new_dir_path_2 = ARTIFACTS_DIR_PATH.joinpath("parsing/diplomas/spbu/work_doc_2")

In [11]:
new_dir_path_1.mkdir(exist_ok=True)
new_dir_path_2.mkdir(exist_ok=True)

In [15]:
for id in tqdm(ids, desc="Iterating through ids"):
    dest = new_dir_path_1 if np.random.random() < 0.5 else new_dir_path_2
    files = old_dir_path.glob(f"{id}.*")
    for file in files:
        subprocess.run(["cp", file.as_posix(), dest.as_posix()])

Iterating through ids:   0%|          | 0/2328 [00:00<?, ?it/s]

Iterating through ids: 100%|██████████| 2328/2328 [00:39<00:00, 58.22it/s]


In [16]:
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc

3.8G	/Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc


In [19]:
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc.tgz

3.4G	/Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc.tgz


In [17]:
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_1

2.0G	/Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1


In [18]:
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_2

2.0G	/Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2


In [20]:
! tar -czvf {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_1.tgz {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_1
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_1.tgz

tar: Removing leading '/' from member names
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/45083.pdf
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/45034.txt
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/44209.docx
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/42334.doc
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/42473.jpg
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/44458.txt
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_1/44470.txt
a Users/nvdenisov2002

In [22]:
! tar -czvf {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_2.tgz {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_2
! du -h {ARTIFACTS_DIR_PATH}/parsing/diplomas/spbu/work_doc_2.tgz

tar: Removing leading '/' from member names
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/40158.txt
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/44989.doc
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/45097.pdf
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/44779.doc
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/40164.txt
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/42015.txt
a Users/nvdenisov2002/study/8-semester/diploma/long_context_LLMs/artifacts/parsing/diplomas/spbu/work_doc_2/44464.txt
a Users/nvdenisov2002/